# Import Required Libraries
Import the necessary libraries, including TensorFlow, Keras, and EfficientNetV2.

In [ ]:
# Import Required Libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

# Load and Preprocess Dataset
Load the skin disease dataset and preprocess the images and labels for training.

In [ ]:
# Load and Preprocess Dataset

# Define the path to the dataset
dataset_path = 'path_to_skin_disease_dataset'

# Create an ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    validation_split=0.2,  # Split the data into training and validation sets
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.2,  # Randomly zoom into images
    shear_range=0.2  # Randomly shear images
)

# Load and preprocess the training data
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # Resize images to 224x224 pixels
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Use the training subset
)

# Load and preprocess the validation data
validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # Resize images to 224x224 pixels
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use the validation subset
)

# Display class indices
class_indices = train_generator.class_indices
print("Class indices:", class_indices)

# Load Pretrained EfficientNetV2 B0 Model
Load the EfficientNetV2 B0 model pretrained on ImageNet.

In [ ]:
# Load Pretrained EfficientNetV2 B0 Model

# Load the EfficientNetV2 B0 model pretrained on ImageNet
base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Create a new model on top of it
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_indices), activation='softmax')  # Number of classes in the dataset
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

# Modify Model for Skin Disease Classification
Modify the pretrained model to adapt it for skin disease classification by adding custom layers.

In [ ]:
# Modify Model for Skin Disease Classification

# Load the EfficientNetV2 B0 model pretrained on ImageNet
base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Create a new model on top of it
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_indices), activation='softmax')  # Number of classes in the dataset
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

# Compile the Model
Compile the modified model with appropriate loss function, optimizer, and metrics.

In [ ]:
# Compile the Model

# Compile the modified model with appropriate loss function, optimizer, and metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Use Adam optimizer with a learning rate of 0.001
    loss='categorical_crossentropy',  # Use categorical crossentropy as the loss function
    metrics=['accuracy']  # Track accuracy as the metric
)

# Train the Model
Train the model using the preprocessed dataset and validate its performance.

In [ ]:
# Train the Model

# Define callbacks for early stopping and saving the best model
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]

# Train the model using the training data and validate using the validation data
history = model.fit(
    train_generator,
    epochs=20,  # Number of epochs to train the model
    validation_data=validation_generator,
    callbacks=callbacks
)

# Plot training and validation accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Evaluate the Model
Evaluate the trained model on the test dataset and report the performance metrics.

In [ ]:
# Evaluate the Model

# Load the test data
test_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # Resize images to 224x224 pixels
    batch_size=32,
    class_mode='categorical',
    subset='validation',  # Use the validation subset as test data
    shuffle=False  # Do not shuffle the test data
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Generate predictions on the test data
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Calculate and display the classification report
from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

# Plot the confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

conf_matrix = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Make Predictions
Use the trained model to make predictions on new skin disease images and visualize the results.

In [ ]:
# Make Predictions

# Load a sample image for prediction
sample_image_path = 'path_to_sample_image.jpg'
sample_image = tf.keras.preprocessing.image.load_img(sample_image_path, target_size=(224, 224))
sample_image_array = tf.keras.preprocessing.image.img_to_array(sample_image)
sample_image_array = np.expand_dims(sample_image_array, axis=0)  # Add batch dimension
sample_image_array = sample_image_array / 255.0  # Rescale pixel values to [0, 1]

# Make a prediction on the sample image
predictions = model.predict(sample_image_array)
predicted_class = np.argmax(predictions, axis=1)[0]
predicted_class_label = class_labels[predicted_class]

# Display the sample image and the prediction result
plt.figure(figsize=(6, 6))
plt.imshow(sample_image)
plt.title(f'Predicted: {predicted_class_label}')
plt.axis('off')
plt.show()